# Anonimización base de Tropa

Dado que la base de tropa contiene información personal de las unidades productivas y sus propietarios, se decide anonimizar la base antes de publicar la metodología del TRIAGE. 

### Definir las rutas de las carpetas

In [1]:
# Se importa la librería para cambiar WD
import os
print(os.getcwd()) # Ver WD antes

# Rutas a las carpetas
root = '/Users/jennyriveraburgos/Library/CloudStorage/GoogleDrive-jrivera.desarrolloeconomico@gmail.com/My Drive/Notebook_Jupyter_TRIAGE/'
code_path = root + os.sep + '0_Code' + os.sep
raw_path = root + os.sep + '1_Raw' + os.sep
temp_path = root + os.sep + '2_Temp' + os.sep
clean_path = root + os.sep + '3_Clean' + os.sep
output_path = root + os.sep + '4_Output' + os.sep

os.chdir(code_path) # Cambia el WD a 0_Code
print(os.getcwd()) # Ver WD después

/Users/jennyriveraburgos/Library/CloudStorage/GoogleDrive-jrivera.desarrolloeconomico@gmail.com/My Drive/Notebook_Jupyter_TRIAGE/0_Code
/Users/jennyriveraburgos/Library/CloudStorage/GoogleDrive-jrivera.desarrolloeconomico@gmail.com/My Drive/Notebook_Jupyter_TRIAGE/0_Code


### Cargar librerías

In [2]:
#Código necesario para instalar librerías
#pip install fontstyle
#pip install prince


# Se importan las librerías externas a usar
import numpy as np
np.random.seed(0)

import pandas as pd
from IPython.display import Markdown as md
import uuid


import Diccionario_anonimizacion as Anon_Triage

### Abrir base de Tropa

Se utilizan solo los datos de las unidades productivas que autorizaron el uso de sus datos.

In [3]:
#file_tropa = '20220919 Datos Tropa v1+v2.csv'
file_tropa = 'Base priorización hasta 21 oct 22.xlsx'

if file_tropa[-5:] == '.xlsx':
    df_tropa = pd.read_excel(raw_path + file_tropa)
else:
    df_tropa = pd.read_csv(raw_path + file_tropa, low_memory=False, sep=',')

n_tropa_total = len(df_tropa)

df_tropa = df_tropa[df_tropa['AUTORIZACION'] == "Si"]
n_tropa_autorizado = len(df_tropa)

In [4]:
md(f"La base de tropa cuenta con un total de {n_tropa_total}. Al dejar sólo las observaciones autorizadas se tienen {n_tropa_autorizado} unidades productivas" )

La base de tropa cuenta con un total de 158016. Al dejar sólo las observaciones autorizadas se tienen 132102 unidades productivas

### Anonimización

Las columnas a anonimizar son las contenidas en la lista col_contacto_anon. La anonimización se hara de la siguiente manera: Nombre de columna a anonimizar + ObjectID

La siguiente imagen es un ejemplo de como queda la base después de la anonimización.

<img src="Anonimizacion.png" width="400" height="200">

In [5]:
col_contacto_anon = ['DIRECCION_COMPLETA', 'COMPLEMENTO',
                'NOMBRE_ENCUESTADO', 'RAZON_SOCIAL', 'NOMBRE_COMERCIAL', 'TELEFONOS', 'CORREOS', 'NOMBRE_PROPIETARIO',
                'NUMERO_ID_PROPIETARIO', 'NUMERO_ID_NEGOCIO', 'TIPO_ID_NEGOCIO']

In [6]:
original_ids = df_tropa['ObjectID'].unique()
DIGITS = 10  
new_ids = {cid: int(uuid.uuid4().hex[:DIGITS], base=16) for cid in original_ids}
df_tropa['ObjectID'] = df_tropa['ObjectID'].map(new_ids)

In [7]:
for col in col_contacto_anon:
    df_tropa[col] = df_tropa.apply(lambda x: col + '_' + str(x['ObjectID']), axis=1)

Adicionalmente, se dejan solo las variables necesarias para la priorización. Estas pueden cambiar según los programas que se incluyen en la priorización.

In [8]:
var_anon = Anon_Triage.var_minimas
var_total = var_anon + col_contacto_anon
df_tropa = df_tropa[var_total]

Se saca una muestra del 20% de la base de tropa para la publicación de la metodología

In [9]:
df_tropa_muestra = df_tropa.sample(frac=0.20)

In [10]:
md(f"La base muestral de tropa cuenta con un total de {len(df_tropa_muestra)}")

La base muestral de tropa cuenta con un total de 26420

In [12]:
df_tropa_muestra.to_excel(raw_path + 'df_tropa_muestra.xlsx', index = False)